In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv

import threading

# Load environment variables and initialize OpenAI client
load_dotenv()
client = OpenAI(
    base_url = "https://openai.vocareum.com/v1",
    api_key=os.getenv("OPENAI_API_KEY"))

# Shared dict for thread-safe collection
agent_outputs = {}

# The shared user prompt
user_prompt = "What are current trends shaping the future of the energy industry?"
print(f"Using parallel agents to answer prompt: {user_prompt}")

# Agent classes using the new OpenAI v1+ SDK
class PolicyAgent:
    def run(self, prompt):
        print(f"Policy Agent resolving prompt: {prompt}")
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a policy expert in global energy policy and climate regulations."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7
        )
        agent_outputs["policy"] = response.choices[0].message.content

class TechnologyAgent:
    def run(self, prompt):
        print(f"Technology Agent resolving prompt: {prompt}")
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are an expert in renewable energy, smart grids, and energy storage technologies."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7
        )
        agent_outputs["tech"] = response.choices[0].message.content

class MarketAgent:
    def run(self, prompt):
        print(f"Market Agent resolving prompt: {prompt}")
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are an energy market analyst focused on global investment, pricing, and demand trends."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7
        )
        agent_outputs["market"] = response.choices[0].message.content

class SummaryAgent:
    def run(self, prompt, inputs):
        combined_prompt = (
            f"The user asked: '{prompt}'\n\n"
            f"Here are the expert responses:\n"
            f"- Policy Expert: {inputs['policy']}\n\n"
            f"- Technology Expert: {inputs['tech']}\n\n"
            f"- Market Expert: {inputs['market']}\n\n"
            "Please summarize the combined insights into a single clear and concise response."
        )
        print(f"Summary Agent resolving prompt: {combined_prompt}")

        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are an energy strategist skilled at synthesizing expert insights."},
                {"role": "user", "content": combined_prompt}
            ],
            temperature=0.7
        )
        return response.choices[0].message.content

# Run the agents
def main():
    policy_agent = PolicyAgent()
    tech_agent = TechnologyAgent()
    market_agent = MarketAgent()
    summary_agent = SummaryAgent()

    threads = [
        threading.Thread(target=policy_agent.run, args=(user_prompt,)),
        threading.Thread(target=tech_agent.run, args=(user_prompt,)),
        threading.Thread(target=market_agent.run, args=(user_prompt,))
    ]

    for t in threads:
        t.start()
    for t in threads:
        t.join()

    final_summary = summary_agent.run(user_prompt, agent_outputs)

    print("\n=== FINAL SUMMARY ===\n")
    print(final_summary)

if __name__ == "__main__":
    main()


Using parallel agents to answer prompt: What are current trends shaping the future of the energy industry?
Policy Agent resolving prompt: What are current trends shaping the future of the energy industry?
Technology Agent resolving prompt: What are current trends shaping the future of the energy industry?
Market Agent resolving prompt: What are current trends shaping the future of the energy industry?
Summary Agent resolving prompt: The user asked: 'What are current trends shaping the future of the energy industry?'

Here are the expert responses:
- Policy Expert: 1. Decarbonization: There is a growing awareness and urgency to reduce carbon emissions and achieve net-zero targets to mitigate climate change impacts. This is driving a shift away from fossil fuels towards renewable sources of energy, such as wind, solar, and hydropower.

2. Electrification: As part of the decarbonization process, electrification of various sectors, including transportation and heating, is expected to incre

In [3]:
import os
import json
import re
from typing import Any, Dict, List
from openai import OpenAI
from dotenv import load_dotenv
import threading

# Load environment variables and initialize OpenAI client
load_dotenv()
client = OpenAI(
    base_url="https://openai.vocareum.com/v1",
    api_key=os.getenv("OPENAI_API_KEY"),
)

MODEL = os.getenv("OPENAI_MODEL", "gpt-4o-mini")

# Shared dict for thread-safe collection of agent outputs
agent_outputs: Dict[str, Any] = {}
_outputs_lock = threading.Lock()

# Example contract text (in a real application, this would be loaded from a file)
contract_text = """
CONSULTING AGREEMENT

This Consulting Agreement (the "Agreement") is made effective as of January 1, 2025 (the "Effective Date"), by and between ABC Corporation, a Delaware corporation ("Client"), and XYZ Consulting LLC, a California limited liability company ("Consultant").

1. SERVICES. Consultant shall provide Client with the following services: strategic business consulting, market analysis, and technology implementation advice (the "Services").

2. TERM. This Agreement shall commence on the Effective Date and shall continue for a period of 12 months, unless earlier terminated.

3. COMPENSATION. Client shall pay Consultant a fee of $10,000 per month for Services rendered. Payment shall be made within 30 days of receipt of Consultant's invoice.

4. CONFIDENTIALITY. Consultant acknowledges that during the engagement, Consultant may have access to confidential information. Consultant agrees to maintain the confidentiality of all such information.

5. INTELLECTUAL PROPERTY. All materials developed by Consultant shall be the property of Client. Consultant assigns all right, title, and interest in such materials to Client.

6. TERMINATION. Either party may terminate this Agreement with 30 days' written notice. Client shall pay Consultant for Services performed through the termination date.

7. GOVERNING LAW. This Agreement shall be governed by the laws of the State of Delaware.

8. LIMITATION OF LIABILITY. Consultant's liability shall be limited to the amount of fees paid by Client under this Agreement.

9. INDEMNIFICATION. Client shall indemnify Consultant against all claims arising from use of materials provided by Client.

10. ENTIRE AGREEMENT. This Agreement constitutes the entire understanding between the parties and supersedes all prior agreements.

IN WITNESS WHEREOF, the parties have executed this Agreement as of the date first above written.
"""

# ---------- Helpers ----------

def _llm_json(system_prompt: str, user_prompt: str, max_tokens: int = 1200) -> Dict[str, Any]:
    """Call the chat completion API requesting JSON. Returns parsed dict or raises."""
    resp = client.chat.completions.create(
        model=MODEL,
        temperature=0,
        response_format={"type": "json_object"},
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        max_tokens=max_tokens,
    )
    content = resp.choices[0].message.content
    return json.loads(content)


def _safe_call_or_fallback(call_fn, fallback_fn):
    try:
        return call_fn()
    except Exception as e:
        # Fallback to deterministic analysis and include the error message for traceability
        fb = fallback_fn()
        fb["_note"] = f"LLM call failed: {type(e).__name__}: {str(e)[:200]}"
        return fb


def _extract_clause_numbers(text: str) -> List[str]:
    # Very simple heuristic to spot numbers like "1.", "2.", etc. used in citations
    nums = re.findall(r"\b([1-9]|10)\.", text)
    return sorted(set(nums), key=lambda x: int(x)) if nums else []


# ---------- Agents ----------

class LegalTermsChecker:
    """Agent that checks for problematic legal terms and clauses in contracts."""
    def run(self, contract_text: str) -> Dict[str, Any]:
        def _call():
            system = (
                "You are a contract analyst. Identify risky legal terms and suggest practical redlines. "
                "Return JSON with keys: issues (array of {clause, finding, risk_level, recommendation, citations}), "
                "overall_risk ('low'|'medium'|'high'). Keep recommendations concise."
            )
            user = (
                "Analyze the following contract text for problematic terms:\n\n"
                f"{contract_text}\n\n"
                "Focus on: termination, indemnification, limitation of liability, IP, confidentiality, governing law, "
                "payment terms, and scope. Produce JSON only."
            )
            return _llm_json(system, user)

        def _fallback():
            issues = []
            # Limitation of Liability – capped at fees paid
            if "LIMITATION OF LIABILITY" in contract_text.upper():
                issues.append({
                    "clause": "8. LIMITATION OF LIABILITY",
                    "finding": "Liability capped to fees paid; no carve-outs (e.g., confidentiality/IP breaches).",
                    "risk_level": "high",
                    "recommendation": "Add carve-outs for confidentiality, IP, willful misconduct, and third-party claims; consider increasing cap (e.g., 2x–3x fees).",
                    "citations": ["8"]
                })
            # Indemnification – one-way
            if "INDEMNIFICATION" in contract_text.upper() and "Client shall indemnify Consultant" in contract_text:
                issues.append({
                    "clause": "9. INDEMNIFICATION",
                    "finding": "One-way indemnity favoring Consultant; no mutual indemnity.",
                    "risk_level": "medium",
                    "recommendation": "Make indemnity mutual, covering each party’s negligence, IP infringement, and misconduct.",
                    "citations": ["9"]
                })
            # Termination – 30-day notice, no cure
            if "TERMINATION" in contract_text.upper():
                issues.append({
                    "clause": "6. TERMINATION",
                    "finding": "30 days’ notice without cure/for-cause detail; no wind-down or transition assistance defined.",
                    "risk_level": "low",
                    "recommendation": "Add for-cause termination with cure periods and define transition assistance and final deliverables.",
                    "citations": ["6"]
                })
            # IP – assignment to Client (generally fine, but consider pre-existing IP)
            if "INTELLECTUAL PROPERTY" in contract_text.upper():
                issues.append({
                    "clause": "5. INTELLECTUAL PROPERTY",
                    "finding": "Full assignment to Client; no carve-out for Consultant’s pre-existing IP or tools.",
                    "risk_level": "low",
                    "recommendation": "Carve out pre-existing IP and tools; grant Client a perpetual, royalty-free license to pre-existing components used.",
                    "citations": ["5"]
                })
            # Confidentiality – generic
            if "CONFIDENTIALITY" in contract_text.upper():
                issues.append({
                    "clause": "4. CONFIDENTIALITY",
                    "finding": "No survival period, exclusions, or security standards specified.",
                    "risk_level": "medium",
                    "recommendation": "Add 2–5 year survival, standard exclusions (public info, independently developed, etc.), and minimum security controls.",
                    "citations": ["4"]
                })
            # Governing law mismatch potential
            if "GOVERNING LAW" in contract_text.upper():
                issues.append({
                    "clause": "7. GOVERNING LAW",
                    "finding": "Delaware governing law; consider conflicts with Consultant’s California presence (labor/IP nuances).",
                    "risk_level": "low",
                    "recommendation": "Add venue, dispute resolution (mediation/arbitration), and acknowledgment of non-conflict with mandatory laws.",
                    "citations": ["7"]
                })
            overall = "high" if any(i["risk_level"] == "high" for i in issues) else ("medium" if any(i["risk_level"] == "medium" for i in issues) else "low")
            return {"issues": issues, "overall_risk": overall}

        return _safe_call_or_fallback(_call, _fallback)


class ComplianceValidator:
    """Agent that validates regulatory and industry compliance of contracts."""
    def run(self, contract_text: str) -> Dict[str, Any]:
        def _call():
            system = (
                "You are a compliance specialist. Check the contract for regulatory/industry compliance concerns. "
                "Assume typical U.S. context. Return JSON with keys: checks (array of {area, status, note, citations}), "
                "gaps (array of strings), overall ('pass'|'caution'|'fail')."
            )
            user = (
                "Validate compliance concerns in this contract:\n\n"
                f"{contract_text}\n\n"
                "Consider confidentiality (PII handling), data protection (CCPA/GDPR if applicable), IP ownership clarity, "
                "payment/recordkeeping, dispute resolution, governing law/venue, independent contractor vs. employee risk, "
                "and audit rights. Produce JSON only."
            )
            return _llm_json(system, user)

        def _fallback():
            checks = []
            checks.append({
                "area": "Data Protection / Confidentiality",
                "status": "caution",
                "note": "No explicit data security, breach notice, or CCPA/GDPR references; add if personal data is processed.",
                "citations": ["4"]
            })
            checks.append({
                "area": "Independent Contractor",
                "status": "caution",
                "note": "No explicit independent contractor status, tax responsibility, or benefits disclaimer; add to reduce misclassification risk.",
                "citations": []
            })
            checks.append({
                "area": "Dispute Resolution / Venue",
                "status": "caution",
                "note": "Governing law exists (Delaware) but no venue/arbitration/mediation; add to reduce litigation uncertainty.",
                "citations": ["7"]
            })
            checks.append({
                "area": "Recordkeeping / Audit",
                "status": "caution",
                "note": "No audit/inspection rights or record retention; consider adding for regulated contexts.",
                "citations": []
            })
            overall = "caution"
            gaps = [
                "Add data security and breach notification terms.",
                "Add independent contractor and tax responsibility language.",
                "Add venue and dispute resolution (e.g., arbitration).",
                "Add audit/inspection rights if required by policy/regulation."
            ]
            return {"checks": checks, "gaps": gaps, "overall": overall}

        return _safe_call_or_fallback(_call, _fallback)


class FinancialRiskAssessor:
    """Agent that assesses financial risks and liabilities in contracts."""
    def run(self, contract_text: str) -> Dict[str, Any]:
        def _call():
            system = (
                "You are a finance/legal ops analyst. Extract financial terms and assess exposure. "
                "Return JSON with keys: terms (object), risks (array of {item, severity, note, citations}), "
                "estimates (object), overall_exposure ('low'|'medium'|'high')."
            )
            user = (
                "Analyze financial exposure in this contract:\n\n"
                f"{contract_text}\n\n"
                "Report: monthly_fee, payment_terms, term_months, early_termination_costs, limitation_of_liability_cap, "
                "indemnity_bias (toward which party), reimbursable_expenses, price_adjustment_mechanism, and any late_fee/interest. "
                "Estimate total_fee_over_term. Produce JSON only."
            )
            return _llm_json(system, user)

        def _fallback():
            # Extract simple heuristics
            monthly_fee = 10000 if "$10,000 per month" in contract_text else None
            term_months = 12 if "period of 12 months" in contract_text else None
            total = (monthly_fee or 0) * (term_months or 0)
            risks = []
            risks.append({
                "item": "Limitation of Liability",
                "severity": "high",
                "note": "Cap equals fees paid; low recovery potential for Client in adverse scenarios.",
                "citations": ["8"]
            })
            risks.append({
                "item": "Indemnification (one-way)",
                "severity": "medium",
                "note": "Client indemnifies Consultant only; no mutual coverage.",
                "citations": ["9"]
            })
            risks.append({
                "item": "Payment Terms",
                "severity": "low",
                "note": "Net 30 specified; no late fee/interest; no expenses clause defined.",
                "citations": ["3"]
            })
            return {
                "terms": {
                    "monthly_fee": monthly_fee,
                    "payment_terms": "Net 30",
                    "term_months": term_months,
                    "early_termination_costs": "Pay through termination date (no penalty specified).",
                    "limitation_of_liability_cap": "Fees paid under the Agreement",
                    "indemnity_bias": "Consultant-favored",
                    "reimbursable_expenses": "Not specified",
                    "price_adjustment_mechanism": "Not specified",
                    "late_fee_interest": "Not specified",
                },
                "risks": risks,
                "estimates": {
                    "total_fee_over_term": total
                },
                "overall_exposure": "high" if total and total > 0 else "medium"
            }

        return _safe_call_or_fallback(_call, _fallback)


class SummaryAgent:
    """Agent that synthesizes findings from all specialized agents."""
    def run(self, contract_text: str, inputs: Dict[str, Any]) -> Dict[str, Any]:
        def _call():
            system = (
                "You are a senior counsel summarizer. Merge agent findings into a concise, executive-ready report. "
                "Return JSON with keys: summary_markdown, key_risks, recommended_redlines, overall_rating."
            )
            user = (
                "Contract text:\n\n"
                f"{contract_text}\n\n"
                "Agent inputs (JSON):\n"
                f"{json.dumps(inputs, indent=2)}\n\n"
                "Synthesize into a practical summary. Keep recommendations specific and actionable."
            )
            return _llm_json(system, user, max_tokens=1600)

        def _fallback():
            # Build a simple markdown synthesis
            legal = inputs.get("legal_terms", {})
            comp = inputs.get("compliance", {})
            fin = inputs.get("financial", {})

            high_flags = []
            for issue in legal.get("issues", []):
                if issue.get("risk_level") == "high":
                    high_flags.append(issue.get("finding", ""))

            for r in fin.get("risks", []):
                if r.get("severity") == "high":
                    high_flags.append(r.get("note", ""))

            recommended = [
                "Add carve-outs to limitation of liability (confidentiality, IP, willful misconduct) and consider higher cap.",
                "Make indemnification mutual with IP infringement coverage.",
                "Add independent contractor, tax responsibility, and data security/breach language.",
                "Add venue and dispute resolution (e.g., arbitration) and define transition assistance on termination.",
                "Clarify reimbursable expenses, late fees/interest, and change order/scope control."
            ]

            overall = "high" if high_flags else "medium"

            md = [
                "## Contract Analysis Summary",
                "**Overall rating:** " + overall.capitalize(),
                "",
                "### Key Risks",
                *([f"- {k}" for k in high_flags] or ["- No high risks detected by fallback analyzer."]),
                "",
                "### Recommended Redlines",
                *[f"- {r}" for r in recommended],
                "",
                "### Financial Snapshot",
                f"- Estimated total over term: ${fin.get('estimates', {}).get('total_fee_over_term', 'N/A')}",
                f"- Liability cap: {fin.get('terms', {}).get('limitation_of_liability_cap', 'N/A')}",
                f"- Indemnity bias: {fin.get('terms', {}).get('indemnity_bias', 'N/A')}"
            ]
            return {
                "summary_markdown": "\n".join(md),
                "key_risks": high_flags,
                "recommended_redlines": recommended,
                "overall_rating": overall
            }

        return _safe_call_or_fallback(_call, _fallback)


# ---------- Router ----------

def analyze_contract(contract_text: str) -> Dict[str, Any]:
    """Run all agents in parallel and summarize their findings."""
    agents = {
        "legal_terms": LegalTermsChecker(),
        "compliance": ComplianceValidator(),
        "financial": FinancialRiskAssessor(),
    }

    def _runner(key: str, agent_obj):
        out = agent_obj.run(contract_text)
        with _outputs_lock:
            agent_outputs[key] = out

    threads: List[threading.Thread] = []
    for key, agent in agents.items():
        t = threading.Thread(target=_runner, args=(key, agent), daemon=True)
        threads.append(t)
        t.start()

    for t in threads:
        t.join()

    # Summarize
    summarizer = SummaryAgent()
    summary = summarizer.run(contract_text, agent_outputs)

    final = {
        "legal_terms": agent_outputs.get("legal_terms"),
        "compliance": agent_outputs.get("compliance"),
        "financial": agent_outputs.get("financial"),
        "summary": summary,
    }
    return final


if __name__ == "__main__":
    print("Enterprise Contract Analysis System")
    print("Analyzing contract...")

    final_analysis = analyze_contract(contract_text)

    print("\n=== FINAL CONTRACT ANALYSIS (Markdown Summary) ===\n")
    print(final_analysis["summary"]["summary_markdown"])

    print("\n=== RAW JSON OUTPUT ===")
    print(json.dumps(final_analysis, indent=2))

Enterprise Contract Analysis System
Analyzing contract...

=== FINAL CONTRACT ANALYSIS (Markdown Summary) ===

# Consulting Agreement Summary

## Overview
The Consulting Agreement between ABC Corporation and XYZ Consulting LLC outlines the terms of engagement for strategic business consulting services. The agreement is set to commence on January 1, 2025, for a duration of 12 months, with a monthly fee of $10,000.

## Key Findings
- **Financial Exposure**: The total fee over the term is estimated at $120,000. Key risks include a limitation of liability that caps Consultant's responsibility at the fees paid, which poses a high risk to the Client.
- **Compliance Gaps**: There are concerns regarding compliance with data protection laws (CCPA/GDPR), absence of a dispute resolution mechanism, and lack of audit rights for the Client.
- **Legal Terms**: Several clauses present risks, including termination notice period, indemnification favoring the Consultant, and vague service definitions.

#